In [2]:
import torch
from torch import nn, optim
from dataloaders import get_dataloaders
from pix2pix import get_models
import train

from importlib import reload
reload(train)

device = torch.device('cuda') if torch.cuda.is_available else 'cpu'

datasets, dataloaders = get_dataloaders(1,1,1)
# Discriminator was 2 layers, testing 4
discriminator, generator = get_models(4,2,16,4, gen_type=2)
discriminator, generator = discriminator.to(device), generator.to(device)
if torch.cuda.device_count() >= 2:
    discriminator = nn.DataParallel(discriminator).to(device)
    generator = nn.DataParallel(generator).to(device)
loss_fn = nn.BCEWithLogitsLoss().to(device)
optim_D = optim.Adam(discriminator.parameters())
optim_G = optim.Adam(generator.parameters(), lr = 0.001)
scheduler_D = optim.lr_scheduler.LambdaLR(optim_D, lambda x: 0.95**x)
scheduler_G = optim.lr_scheduler.LambdaLR(optim_G, lambda x: 0.95**x)


(discriminator, generator, acc_true, acc_fake,
 loss_D, loss_G, dose_score, best_idx) = train.train_GAN(dataloaders, discriminator,
                                                         generator, loss_fn, optim_D, 
                                                         optim_G, scheduler_D, scheduler_G,
                                                         patch = 35,
                                                         n_epochs = 50, lambda_pixelwise=0)

Epoch  1/50
Train loss: D 0.2625, G 2.0084
Val loss  : D 1.5220, G 1.4984
Train acc: Real 0.9449, Fake 0.9553
Val acc  : Real 0.0000, Fake 0.6613
Dose score: Train 209.3275, Val 101.8364
Epoch time = 170s
Elapsed time = 170s
--------------------

Epoch  2/50
Train loss: D 0.2718, G 2.9511
Val loss  : D 1.4376, G 4.6359
Train acc: Real 0.9154, Fake 0.9331
Val acc  : Real 0.0052, Fake 0.9940
Dose score: Train 68.5910, Val 17.3755
Epoch time = 155s
Elapsed time = 325s
--------------------

Epoch  3/50
Train loss: D 0.1548, G 4.2550
Val loss  : D 1.2556, G 1.9377
Train acc: Real 0.9542, Fake 0.9595
Val acc  : Real 0.0252, Fake 0.9754
Dose score: Train 130.7369, Val 114.4910
Epoch time = 395s
Elapsed time = 719s
--------------------

Epoch  4/50
Train loss: D 0.5669, G 1.6014
Val loss  : D 0.7788, G 1.7920
Train acc: Real 0.7028, Fake 0.6962
Val acc  : Real 0.0196, Fake 0.9653
Dose score: Train 39.3430, Val 43.6202
Epoch time = 361s
Elapsed time = 1080s
--------------------

Epoch  5/50
Tra

Epoch 35/50
Train loss: D 0.6295, G 0.8448
Val loss  : D 0.7099, G 1.0665
Train acc: Real 0.6989, Fake 0.6872
Val acc  : Real 0.0996, Fake 0.9374
Dose score: Train 15.9508, Val 14.2599
Epoch time = 156s
Elapsed time = 6269s
--------------------

Epoch 36/50
Train loss: D 0.6705, G 0.7750
Val loss  : D 0.7511, G 0.6659
Train acc: Real 0.6141, Fake 0.6066
Val acc  : Real 0.4658, Fake 0.4442
Dose score: Train 16.1803, Val 14.6825
Epoch time = 154s
Elapsed time = 6423s
--------------------

Epoch 37/50
Train loss: D 0.6548, G 0.7884
Val loss  : D 0.9202, G 1.2487
Train acc: Real 0.6686, Fake 0.6484
Val acc  : Real 0.0955, Fake 0.9345
Dose score: Train 15.9735, Val 15.5618
Epoch time = 155s
Elapsed time = 6578s
--------------------

Epoch 38/50
Train loss: D 0.6520, G 0.7973
Val loss  : D 0.8046, G 0.6436
Train acc: Real 0.6438, Fake 0.6601
Val acc  : Real 0.1577, Fake 0.3776
Dose score: Train 16.7922, Val 17.3530
Epoch time = 155s
Elapsed time = 6733s
--------------------

Epoch 39/50
Trai

In [3]:
import os
lr, batch, lam = 0.001, 1, 0
path = f'saved_items/{lr}-{batch}-{lam}'

if not os.path.isdir(path):
    os.makedirs(path)
    
for item in ['acc_true', 'acc_fake',
             'loss_D', 'loss_G', 'dose_score', 'best_idx']:
        
    torch.save(eval(item), f'{path}/{item}.pt')
torch.save(discriminator.state_dict(), f'{path}/discriminator_dict.pt')
torch.save(generator.state_dict(), f'{path}/generator_dict.pt')
